In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [2]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data"

In [13]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [14]:
query_productos_soi = """ 
WITH CLIENTES_ACTIVOS AS (
   SELECT  BA_BCO_FFON, BACVE_BANC, BANOM_BAN
   FROM SOI.SOI_BANCO sb
   WHERE BA_ACTIVO = 'S'
), CLT_LINEA AS (
	SELECT CA.*, SBL.BLNO_LINEA
	FROM CLIENTES_ACTIVOS CA
	LEFT JOIN SOI.SOI_BANCO_LIN SBL ON CA.BACVE_BANC = SBL.BLCVE_BANC
), LINEA_CT1 AS (
	SELECT SL.*, SB.TFDESC_CONT 
	FROM SOI.SOI_LINEA sl
	LEFT JOIN SOI.SOI_TIP_CREDI SB ON SL.LITIP_CONT = SB.TFTIP_CONT
), CLT_LINEA_DESCRIP AS (
	SELECT CL.*, SL.*
	FROM CLT_LINEA CL 
	LEFT JOIN LINEA_CT1 SL ON CL.BLNO_LINEA = SL.LINO_LINEA
) SELECT * FROM CLT_LINEA_DESCRIP
"""

In [15]:
path_dir_productos = get_table(path_dir_today, db_soi, "productos_clientes_soi", query = query_productos_soi)

# Transform the data and output the transformation

In [35]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SOI/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SOI/agrupados/22_04_2025'

In [3]:
path_dir_output_transformation = '/home/ale1726/proyects/datalake/clientes/data/productos/SOI/agrupados/22_04_2025'

In [4]:
path_dir_productos = '/home/ale1726/proyects/datalake/clientes/data/productos/SOI/data/22_04_2025/productos_clientes_soi.dat'

In [5]:
productos_soi =  pd.read_csv(path_dir_productos, low_memory=False)

In [6]:
productos_soi.columns

Index(['BA_BCO_FFON', 'BACVE_BANC', 'BANOM_BAN', 'BLNO_LINEA', 'LINO_LINEA',
       'LITIPO_LIN', 'LIF_FIRMA', 'LIF_EFECT', 'LIMTO_LIN', 'LIMONEDA',
       'LIINT_MOR1', 'LIINT_MOR2', 'LIPLA_AMO1', 'LIPLA_AMO2', 'LIVIGENCIA',
       'LIFECHAREN', 'LIGASTLOCA', 'LIDES_GTO1', 'LIDES_GTO2', 'LINUM_SHCP',
       'LIF_SHCP', 'LIOBLISHCP', 'LICOMXLIN', 'LIACUM_DES', 'LIF_LIM_DE',
       'LITIP_CONT', 'LIPLA_AMO3', 'LIPERIOCAP', 'LIPERIOINT', 'LICVE_CORR',
       'LIISR', 'LIINT_FIN', 'LICVE_ACRE', 'LIREF_CRED', 'LIMTO_REF',
       'LISTATUS', 'LICVE_GAR', 'LICVE_DOM', 'LICTA_DOM', 'LINUEVO',
       'LIACREDIT', 'LIE_ACREDI', 'LIDIA_FER1', 'LIDIA_FER2', 'LIF_PRPACAP',
       'LIF_LIQUIDACION', 'LIPLA_AMO4', 'LI_PL_ID_ACRE', 'LI_PL_ID_GAR',
       'LI_PL_ID_DOM', 'LICVE_INTER', 'LI_PL_ID_INTER', 'LIMTO_CAP',
       'TFDESC_CONT'],
      dtype='object')

In [17]:
productos_soi["LIF_FIRMA"] = pd.to_datetime(productos_soi["LIF_FIRMA"])
productos_soi["AÑO_FFIRMA"] = productos_soi["LIF_FIRMA"].dt.year.astype('Int64')
productos_soi["MES_FFIRMA"] = productos_soi["LIF_FIRMA"].dt.month
productos_soi["MES_STR"] = productos_soi["LIF_FIRMA"].dt.month_name()  
productos_soi["LIVIGENCIA"] = pd.to_datetime(productos_soi["LIVIGENCIA"])

In [44]:
agrupado_portafolio = (
    productos_soi.groupby(['TFDESC_CONT'], as_index=False)
    .agg(
        valor=('LINO_LINEA', 'count'), 
    ).sort_values(by=['TFDESC_CONT'], ascending=True)
)

In [47]:
agrupado_portafolio

,TFDESC_CONT,valor
0,CAPITAL DE TRABAJO,20
1,COLOCACIONES PRIVADAS,2
2,COLOCACIONES PUBLICAS,1
3,COLOCACIONES PUBLICAS EXTERNAS,5
4,CREDITO AVALADO,3
5,CREDITO ESPECIFICO,13
6,CREDITO SINDICADO,2
7,EMISION DE EUROPAGARES,4
8,EUROPAPEL COMERCIAL,3
9,LINEA GLOBAL,51


In [46]:
agrupado_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_portafolio.csv"), index=False)

In [ ]:
agrupado_operaciones_anuales = (
    productos_soi.groupby(['AÑO_FFIRMA'], as_index=False)
    .agg(
        num_contratos=('LINO_LINEA', 'count'), 
        monto_anuales_sum=('LIMTO_LIN', lambda x: round(x.sum(), 2)),
        monto_anuales_min=('LIMTO_LIN', 'min'), 
        monto_anuales_max=('LIMTO_LIN', 'max'), 
        monto_anuales_mean=('LIMTO_LIN', lambda x: round(x.mean(), 2)),
        monto_anuales_median=('LIMTO_LIN', 'median')
    ).sort_values(by=['AÑO_FFIRMA'], ascending=True)
)

In [39]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [41]:
agrupado_operaciones_anuales_mes = (
    productos_soi.groupby(['AÑO_FFIRMA','MES_FFIRMA','MES_STR'], as_index=False)
    .agg(
        num_contratos=('LINO_LINEA', 'count'), 
        monto_anuales_sum=('LIMTO_LIN', lambda x: round(x.sum(), 2)),
        monto_anuales_min=('LIMTO_LIN', 'min'), 
        monto_anuales_max=('LIMTO_LIN', 'max'), 
        monto_anuales_mean=('LIMTO_LIN', lambda x: round(x.mean(), 2)),
        monto_anuales_median=('LIMTO_LIN', 'median')
    ).sort_values(by=['AÑO_FFIRMA','MES_FFIRMA'], ascending=True)
)

In [42]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)

In [9]:
productos_soi.columns

Index(['BA_BCO_FFON', 'BACVE_BANC', 'BANOM_BAN', 'BLNO_LINEA', 'LINO_LINEA',
       'LITIPO_LIN', 'LIF_FIRMA', 'LIF_EFECT', 'LIMTO_LIN', 'LIMONEDA',
       'LIINT_MOR1', 'LIINT_MOR2', 'LIPLA_AMO1', 'LIPLA_AMO2', 'LIVIGENCIA',
       'LIFECHAREN', 'LIGASTLOCA', 'LIDES_GTO1', 'LIDES_GTO2', 'LINUM_SHCP',
       'LIF_SHCP', 'LIOBLISHCP', 'LICOMXLIN', 'LIACUM_DES', 'LIF_LIM_DE',
       'LITIP_CONT', 'LIPLA_AMO3', 'LIPERIOCAP', 'LIPERIOINT', 'LICVE_CORR',
       'LIISR', 'LIINT_FIN', 'LICVE_ACRE', 'LIREF_CRED', 'LIMTO_REF',
       'LISTATUS', 'LICVE_GAR', 'LICVE_DOM', 'LICTA_DOM', 'LINUEVO',
       'LIACREDIT', 'LIE_ACREDI', 'LIDIA_FER1', 'LIDIA_FER2', 'LIF_PRPACAP',
       'LIF_LIQUIDACION', 'LIPLA_AMO4', 'LI_PL_ID_ACRE', 'LI_PL_ID_GAR',
       'LI_PL_ID_DOM', 'LICVE_INTER', 'LI_PL_ID_INTER', 'LIMTO_CAP',
       'TFDESC_CONT', 'AÑO_FFIRMA', 'MES_FFIRMA', 'MES_STR'],
      dtype='object')

In [ ]:
productos_soi

In [18]:
agrupado_clientes_año = (
    productos_soi.groupby(['BACVE_BANC','BANOM_BAN', 'AÑO_FFIRMA'], as_index=False)
    .agg(
        transacciones=('LINO_LINEA', 'count'),
        tipo_de_credito = ('TFDESC_CONT', 'nunique'),
        monto_mean=('LIMTO_LIN', lambda x: round(x.mean(), 2) if x.mean() != 0 else 0)
        
    )
    .sort_values(by=['AÑO_FFIRMA'], ascending=False)
)

In [24]:
agrupado_clientes_año

,BACVE_BANC,BANOM_BAN,AÑO_FFIRMA,transacciones,tipo_de_credito,monto_mean
17,1520,KREDITANSTALT FUR WIEDERAUFBAU (KFW) (A),2021,2,1,1.127300e+08
48,47881,EUROPEAN INVESTMENT BANK,2020,1,1,1.500000e+08
39,10619,MITSUBISHI BANK LTD,2019,2,1,1.000000e+08
38,10619,MITSUBISHI BANK LTD,2018,1,1,1.000000e+08
16,1520,KREDITANSTALT FUR WIEDERAUFBAU (KFW) (A),2018,3,1,4.914000e+07
0,95,CORPORACION ANDINA DE FOMENTO (CAF),2017,2,1,2.500000e+07
49,47993,BNYM LONDON,2017,2,1,1.250000e+10
47,47881,EUROPEAN INVESTMENT BANK,2017,1,1,1.000000e+08
37,10619,MITSUBISHI BANK LTD,2017,2,1,5.000000e+07
36,10619,MITSUBISHI BANK LTD,2016,5,1,4.300000e+07


In [23]:
agrupado_clientes_año.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_año.csv"), index=False)